In [0]:
%tensorflow_version 1.x 
import keras
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D


In [72]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split

In [0]:
data1='/content/drive/My Drive/fintest/1coal'
data2='/content/drive/My Drive/fintest/2coal'
data3='/content/drive/My Drive/fintest/3coal'

In [0]:
def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,0,1])])
        
    shuffle(images)
    return images

In [76]:
data = data_with_label()

100%|██████████| 465/465 [00:13<00:00, 33.55it/s]


In [0]:
import random
random.shuffle(data)
train_data, test_data = train_test_split(data, test_size=0.2)

In [0]:
from keras.applications.resnet50 import preprocess_input
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1,224,224,3)
tr_img_data = preprocess_input(tr_img_data)
tr_lbl_data = np.array([i[1] for i in train_data])
tst_img_data = np.array([i[0] for i in test_data]).reshape(-1,224,224,3)
tst_img_data = preprocess_input(tst_img_data)
tst_lbl_data = np.array([i[1] for i in test_data])

In [79]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [80]:
import pandas as pd
restnet.trainable = True
set_trainable = False
for layer in restnet.layers:
    if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in restnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_7,False
1,<keras.layers.convolutional.ZeroPadding2D obje...,conv1_pad,False
2,<keras.layers.convolutional.Conv2D object at 0...,conv1,False
3,<keras.layers.normalization.BatchNormalization...,bn_conv1,False
4,<keras.layers.core.Activation object at 0x7ff9...,activation_295,False
...,...,...,...
171,<keras.layers.convolutional.Conv2D object at 0...,res5c_branch2c,True
172,<keras.layers.normalization.BatchNormalization...,bn5c_branch2c,True
173,<keras.layers.merge.Add object at 0x7ff751163940>,add_112,True
174,<keras.layers.core.Activation object at 0x7ff7...,activation_343,True


In [81]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_finetuned = Sequential()
model_finetuned.add(restnet)
model_finetuned.add(Dense(512, activation='relu', input_dim=(224,224,1)))
model_finetuned.add(Dropout(0.5))
model_finetuned.add(Dense(3, activation='sigmoid'))
model_finetuned.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])
model_finetuned.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_6 (Model)              (None, 100352)            23587712  
_________________________________________________________________
dense_14 (Dense)             (None, 512)               51380736  
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 1539      
Total params: 74,969,987
Trainable params: 54,797,827
Non-trainable params: 20,172,160
_________________________________________________________________


In [83]:
import keras 
checkpoint_path = "/content/drive/My Drive/cpr133.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
history = model_finetuned.fit(tr_img_data, 
          tr_lbl_data,  
          epochs=10,
          validation_data=(tst_img_data,tst_lbl_data),
          callbacks=[cp_callback])  # Pass callback to training

Train on 923 samples, validate on 231 samples
Epoch 1/10
923/923 [==============================] - 12s 13ms/step - loss: 0.9138 - acc: 0.6067 - val_loss: 0.6493 - val_acc: 0.6926

Epoch 00001: saving model to /content/drive/My Drive/cpr133.ckpt
Epoch 2/10
923/923 [==============================] - 8s 9ms/step - loss: 0.2938 - acc: 0.8971 - val_loss: 0.5952 - val_acc: 0.7316

Epoch 00002: saving model to /content/drive/My Drive/cpr133.ckpt
Epoch 3/10
923/923 [==============================] - 8s 9ms/step - loss: 0.1285 - acc: 0.9491 - val_loss: 0.4727 - val_acc: 0.8052

Epoch 00003: saving model to /content/drive/My Drive/cpr133.ckpt
Epoch 4/10
923/923 [==============================] - 8s 9ms/step - loss: 0.0643 - acc: 0.9783 - val_loss: 0.4168 - val_acc: 0.8225

Epoch 00004: saving model to /content/drive/My Drive/cpr133.ckpt
Epoch 5/10
923/923 [==============================] - 8s 9ms/step - loss: 0.0389 - acc: 0.9881 - val_loss: 0.5870 - val_acc: 0.7922

Epoch 00005: saving model t